In [ ]:
from evoframe.reward_builders import RewardBuilderGame
from evoframe.population_update_builders import PopulationUpdateBuilderStatic
from evoframe.selector_function import SelectorFunctionFactory
from evoframe import EvolutionBuilder
from evoframe.models import FeedForwardNetwork
from evoframe.models import ActivationFunctions
from evoframe.games import Game
from evoframe import get_agent_wrapper_func

import numpy as np
import pandas as pd

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from evoframe.experiment_results import *
from evoframe.utility import clean_experiment_directory

%matplotlib inline

# Guess the point

In [ ]:
from evoframe.games import GuessPoint

In [ ]:
# Game
game_creation_func = lambda context: GuessPoint(np.array([0.2,0.8,0.5]), np.array([0.4, 0.5, 10]))

# Game-Model interface
predict_func = lambda model, game: model.predict(game.inputs)
agent_wrapper_func = get_agent_wrapper_func(predict_func)

# Reward function and update env function
reward_func, get_context_func = RewardBuilderGame() \
.with_game_creation_func(game_creation_func) \
.with_agent_wrapper_func(agent_wrapper_func) \
.get()

# Model
layer_sizes = [3, 2, 3]
get_model_func = lambda: FeedForwardNetwork(layer_sizes, ActivationFunctions.get_relu(), ActivationFunctions.get_id(), with_bias=False)

# Update population function
get_new_pop_func = PopulationUpdateBuilderStatic() \
.add_operator("es_1_copy", 0.1) \
.add_operator("es_n_rewards_gradient_and_mutation", 0.1, 0.01, 0.1, 1) \
.add_operator("es_n_rewards_gradient_and_mutation", 0.1, 0.01, 0.2, 1) \
.add_operator("es_n_rewards_gradient_and_mutation", 0.1, 0.01, 0.3, 1) \
.add_operator("es_n_rewards_gradient_and_mutation", 0.1, 0.05, 0.1, 1) \
.add_operator("es_n_rewards_gradient_and_mutation", 0.1, 0.05, 0.2, 1) \
.add_operator("es_n_rewards_gradient_and_mutation", 0.1, 0.05, 0.3, 1) \
.add_operator("es_n_rewards_gradient_and_mutation", 0.1, 0.1, 0.1, 1) \
.add_operator("es_n_rewards_gradient_and_mutation", 0.1, 0.1, 0.2, 1) \
.add_operator("es_n_rewards_gradient_and_mutation", 0.1, 0.1, 0.3, 1) \
.add_operator("es_n_rewards_gradient", 0.01) \
.add_selector_func(SelectorFunctionFactory.get_geometric_selector_function(0.3)) \
.get()

# Evolution function
evolution_func = EvolutionBuilder() \
    .with_get_model_func(get_model_func) \
    .with_reward_func(reward_func) \
    .with_get_new_pop_func(get_new_pop_func) \
    .with_get_context_func(get_context_func) \
    .get()

experiment_name = "guesspoint"
pop_size = 200
num_epochs = 10
clean_experiment_directory(experiment_name)
evolution_func(experiment_name, pop_size, num_epochs, num_threads=8)

In [ ]:
plot_rewards(experiment_name, add_scatter=True)

In [ ]:
interact(show_best_fnn_weights,
         experiment_name=fixed(experiment_name),
         epoch=widgets.IntSlider(min=1, max=num_epochs, step=1, value=1))

In [ ]:
inp = np.array([0.2,0.8,0.5])

def get_random_input_func():
    global inp
    return inp

In [ ]:
plot_behavioural_differences(experiment_name, get_random_input_func, mode="first_best", iterations=1)

In [ ]:
plot_behavioural_differences(experiment_name, get_random_input_func, mode="last_best", iterations=1)

In [ ]:
plot_params_similarity(experiment_name, only_best=False)

In [ ]:
plot_params_similarity(experiment_name, only_best=True)

In [ ]:
plot_params_statistics(experiment_name)

# Guess the function

In [ ]:
from evoframe.games import GuessFunction

In [ ]:
# Game
game_func = lambda i: np.array([2*i[0]-3*i[1]+4, i[1]-8*i[2]-5])
input_dim = 3
input_domains = [(-1,1),(-1,1),(3,7)]
sample_every = [0.1, 0.1, 0.3]
game_creation_func = lambda context: GuessFunction(game_func, input_dim, input_domains, sample_every)

# Model
layer_sizes = [3, 5, 2]
get_model_func = lambda: FeedForwardNetwork(layer_sizes, ActivationFunctions.get_arctan(), ActivationFunctions.get_id())

# Game-Model interface
predict_func = lambda model, inputs: model.predict(inputs)
agent_wrapper_func = get_agent_wrapper_func(predict_func)

# Reward function and update env function
reward_func, get_context_func = RewardBuilderGame() \
.with_game_creation_func(game_creation_func) \
.with_agent_wrapper_func(agent_wrapper_func) \
.get()

# Update population function
get_new_pop_func = PopulationUpdateBuilderStatic() \
.add_operator("es_1_mutation", 0.2, 0.1, 0.1) \
.add_operator("es_1_mutation", 0.2, 0.1, 0.5) \
.add_operator("es_1_mutation", 0.2, 0.3, 0.3) \
.add_operator("es_1_mutation", 0.2, 0.5, 0.5) \
.add_operator("es_1_mutation", 0.2, 0.5, 0.1) \
.add_operator("es_2_crossover", 0.1, 0.8) \
.add_operator("es_1_copy", 0.1) \
.add_operator("es_n_rewards_gradient", 0.1) \
.add_selector_func(SelectorFunctionFactory.get_geometric_selector_function(0.3)) \
.get()

# Evolution function
evolution_func = EvolutionBuilder() \
    .with_get_model_func(get_model_func) \
    .with_reward_func(reward_func) \
    .with_get_new_pop_func(get_new_pop_func) \
    .with_get_context_func(get_context_func) \
    .get()

pop_size = 40
num_epochs = 100
experiment_name = "guessfunction"
clean_experiment_directory(experiment_name)
evolution_func(experiment_name, pop_size, num_epochs, num_threads=8)

In [ ]:
plot_rewards(experiment_name)

In [ ]:
test_array = np.array([0.5,0.5,5])
pickle_load_best_model_of_epoch(experiment_name, num_epochs, pop_size).predict(test_array), game_func(test_array)

In [ ]:
interact(show_best_fnn_weights,
         experiment_name=fixed(experiment_name),
         epoch=widgets.IntSlider(min=1, max=num_epochs, step=1, value=1))

In [ ]:
plot_params_statistics(experiment_name)

In [ ]:
input_domains = [(-1,1),(-1,1),(3,7)]
sample_every = [0.1, 0.1, 0.3]

def get_random_input_func():
    global input_domains
    global sample_every
    inp = []
    for i,input_domain in enumerate(input_domains):
        point_i = np.random.randint(low=0, high=int((input_domain[1] - input_domain[0])/sample_every[i]))
        point = input_domain[0] + sample_every[i] * point_i
        inp.append(point)
    inp = np.array(inp)
    return inp

In [ ]:
plot_behavioural_differences(experiment_name, get_random_input_func, mode="first_best")

In [ ]:
plot_behavioural_differences(experiment_name, get_random_input_func, mode="last_best")

In [ ]:
plot_behavioural_variances_to_input(experiment_name, get_random_input_func)

In [ ]:
plot_params_similarity(experiment_name, only_best=False, iterations=600)

In [ ]:
plot_params_similarity(experiment_name, only_best=True, iterations=600)

# Other

In [ ]:
import imageio

def to_gif(experiment_name, num_epochs, duration=0.5):
    for i in range(1, num_epochs+1):
        print(i)
        f = show_best_fnn_weights(experiment_name, i)
        b = f.to_image(format="png")
        with open("images/image{}.png".format(i), "wb") as fi:
            fi.write(b)
    images = []
    filenames = ["images/image{}.png".format(i) for i in range(1, num_epochs+1)]
    for i,filename in enumerate(filenames):
        print(i)
        images.append(imageio.imread(filename))
    imageio.mimsave('nn.gif', images, duration=duration)